In [38]:
import pandas as pd

# Asumiendo que tu archivo se llama 'classification.xlsx'
df = pd.read_csv('c:/Users/Matias/OneDrive/Escritorio/Tesis/Data Portugal/BCDR-F01_dataset/BCDR-F01_dataset/bcdr_f01_outlines.csv')
df

,patient_id,study_id,series,lesion_id,segmentation_id,image_view,image_filename,lw_x_points,lw_y_points,mammography_type,mammography_nodule,mammography_calcification,mammography_microcalcification,mammography_axillary_adenopathy,mammography_architectural_distortion,mammography_stroma_distortion,age,density,classification
0,179,185,1,185,3,2,patient_179/study_185/img_179_185_1_LCC.tif,12 13 14 14 14 14 14 15 16 17 18 18 19 20 21 ...,603 602 601 600 599 598 597 596 595 594 593 5...,1,0,0,1,0,0,1,53,1,Malign
1,179,185,1,185,4,4,patient_179/study_185/img_179_185_1_LO.tif,92 93 94 95 96 97 98 99 100 101 102 103 104 1...,835 835 834 834 833 833 833 833 833 833 833 8...,1,0,0,1,0,0,1,53,1,Malign
2,182,188,1,188,7,2,patient_182/study_188/img_182_188_1_LCC.tif,424 425 426 427 428 429 430 431 432 433 434 4...,278 278 278 278 277 276 275 274 273 273 273 2...,1,1,0,0,0,0,0,68,1,Malign
3,182,188,1,188,8,4,patient_182/study_188/img_182_188_1_LO.tif,517 518 519 520 521 522 523 524 525 526 527 5...,413 413 413 412 411 410 410 410 410 410 410 4...,1,1,0,0,0,0,0,68,1,Malign
4,190,196,1,196,13,2,patient_190/study_196/img_190_196_1_LCC.tif,182 183 184 185 186 187 188 189 190 191 192 1...,520 520 520 520 520 520 519 519 519 519 519 5...,1,1,0,0,0,0,0,57,3,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,596,650,1,621,1424,2,patient_596/study_650/img_596_650_1_LCC.tif,314 313 312 311 310 309 308 307 306 305 304 3...,858 858 858 858 858 858 858 858 858 859 859 8...,1,1,0,0,0,0,0,59,1,Malign
358,615,672,1,631,1453,4,patient_615/study_672/img_615_672_1_LO.tif,80 79 79 79 79 78 77 76 75 75 74 73 72 71 71 ...,627 628 629 630 631 632 633 634 635 636 637 6...,1,1,0,1,0,0,0,71,3,Malign
359,615,672,1,631,1454,2,patient_615/study_672/img_615_672_1_LCC.tif,71 70 70 70 70 69 68 67 67 67 67 67 67 66 65 ...,580 581 582 583 584 585 586 587 588 589 590 5...,1,1,0,1,0,0,0,71,3,Malign
360,681,740,1,1056,1494,2,patient_681/study_740/img_681_740_1_LCC.tif,206 205 204 203 203 202 201 200 199 198 197 1...,465 466 467 468 469 470 471 472 473 474 475 4...,1,1,0,0,0,0,0,51,3,Malign


# Funcion


In [26]:
import pandas as pd
import os
import shutil
from pathlib import Path
from PIL import Image
import tensorflow as tf

def organizar_imagenes(patient_dir, organized_dir, csv_file_path, pacientes_procesados):
    df = pd.read_csv(csv_file_path)
    
    if not os.path.exists(organized_dir):
        os.makedirs(organized_dir)

    for patient_folder in os.listdir(patient_dir):
        patient_id = patient_folder.split('_')[1]

        if patient_id in pacientes_procesados:
            continue

        pacientes_procesados.add(patient_id)
        try:
            classification = df[df['patient_id'] == int(patient_id)]['classification'].values[0].strip().lower()
        except IndexError:
            print(f"No se encontró clasificación para el paciente {patient_id}")
            continue

        patient_path = os.path.join(patient_dir, patient_folder)
        
        for study_folder in os.listdir(patient_path):
            study_path = os.path.join(patient_path, study_folder)
            
            for image_file in os.listdir(study_path):
                new_image_name = f"{patient_id}_{classification}_{image_file}"
                destination_path = os.path.join(organized_dir, new_image_name)
                
                shutil.copy(os.path.join(study_path, image_file), destination_path)

def convertir_tif_a_jpg(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".tif"):
                img_path = os.path.join(root, file)
                img = Image.open(img_path)
                output_path = os.path.join(output_dir, os.path.basename(file).replace(".tif", ".jpg"))
                img.save(output_path, "JPEG")

def organizar_imagenes_por_clase(base_dir):
    if not os.path.exists(base_dir):
        print(f"El directorio {base_dir} no existe.")
        return

    files = [f for f in os.listdir(base_dir) if os.path.isfile(os.path.join(base_dir, f))]
    
    for file in files:
        if "_" in file and (file.endswith('.jpg') or file.endswith('.png')):
            parts = file.split('_')
            if len(parts) > 2:
                # Elimina los espacios en blanco y convierte el nombre de la clase a minúsculas
                class_name = parts[1].strip().lower()
                
                class_dir = os.path.join(base_dir, class_name)
                if not os.path.exists(class_dir):
                    os.makedirs(class_dir)
                
                src_path = os.path.join(base_dir, file)
                dst_path = os.path.join(class_dir, file)
                shutil.move(src_path, dst_path)
            else:
                print(f"No se puede determinar la clase para el archivo: {file}")

def crear_y_normalizar_dataset(output_dir):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        output_dir,
        labels='inferred',
        label_mode='int',
        image_size=(256, 256),
        batch_size=32)

    normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
    normalized_ds = dataset.map(lambda x, y: (normalization_layer(x), y))
    return normalized_ds

def procesar_datasets(ruta_carpeta_base, organized_dir):
    ruta_carpeta_base = Path(ruta_carpeta_base)
    pacientes_procesados = set()

    for dataset_dir in ruta_carpeta_base.iterdir():
        if dataset_dir.is_dir() and "BCDR" in dataset_dir.name and "bcdr_dn01" not in dataset_dir.name.lower():
            patient_dir = dataset_dir / 'patients'
            csv_file_name = f"{dataset_dir.name.replace('-', '_').lower().replace('_dataset', '')}_outlines.csv"
            csv_file_path = dataset_dir / csv_file_name
            print(f"Procesando {dataset_dir.name} usando {csv_file_path}")
            organizar_imagenes(patient_dir, organized_dir, csv_file_path, pacientes_procesados)

# Configuraciones iniciales
ruta_carpeta_base = 'C:/Users/Matias/OneDrive/Escritorio/Tesis/Data Portugal'
organized_dir = 'C:/Users/Matias/OneDrive/Escritorio/Tesis/Data Portugal/organized_images'
output_dir = 'C:/Users/Matias/OneDrive/Escritorio/Tesis/Data Portugal/converted_images'

# Ejecutar el proceso completo
procesar_datasets(ruta_carpeta_base, organized_dir)
convertir_tif_a_jpg(organized_dir, output_dir)
organizar_imagenes_por_clase(output_dir)
dataset_final = crear_y_normalizar_dataset(output_dir)


Procesando BCDR-D01_dataset usando C:\Users\Matias\OneDrive\Escritorio\Tesis\Data Portugal\BCDR-D01_dataset\bcdr_d01_outlines.csv
Procesando BCDR-F01_dataset usando C:\Users\Matias\OneDrive\Escritorio\Tesis\Data Portugal\BCDR-F01_dataset\bcdr_f01_outlines.csv
Procesando BCDR-F02_dataset usando C:\Users\Matias\OneDrive\Escritorio\Tesis\Data Portugal\BCDR-F02_dataset\bcdr_f02_outlines.csv
Procesando BCDR-F03_dataset usando C:\Users\Matias\OneDrive\Escritorio\Tesis\Data Portugal\BCDR-F03_dataset\bcdr_f03_outlines.csv
Found 1293 files belonging to 2 classes.


In [27]:
dataset_final

<MapDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

# Red neuronal


In [28]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import random
import cv2

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directorio con todas las imágenes organizadas por carpetas de clase.
            transform (callable, optional): Transformaciones opcionales a aplicar a las imágenes.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []

        # Cargar las imágenes y etiquetas
        for label, class_dir in enumerate(sorted(os.listdir(root_dir))):
            class_dir_path = os.path.join(root_dir, class_dir)
            for image_name in os.listdir(class_dir_path):
                self.images.append(os.path.join(class_dir_path, image_name))
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [29]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Ajusta según el tamaño de tus imágenes
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalización para modelos preentrenados
])

# Cambia 'your_data_directory' al directorio donde tienes tus imágenes organizadas
dataset = dataset_final

# Dividir el dataset en entrenamiento y validación
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [30]:
class CNNModel(nn.Module):
    def __init__(self, n_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        # Ajusta la siguiente línea según el tamaño de tus imágenes después del pooling
        self.fc1 = nn.Linear(32 * 64 * 64, 128)  # Ajuste si tu tamaño de imagen es diferente
        self.fc2 = nn.Linear(128, n_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 64 * 64)  # Ajuste si tu tamaño de imagen es diferente
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Asume n_classes como el número de subdirectorios/clases en tu dataset
n_classes = len(os.listdir('c:/Users/Matias/OneDrive/Escritorio/Tesis/Data Portugal/converted_images'))
model = CNNModel(n_classes=n_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [33]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Calcula la pérdida y precisión de validación después de cada época
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            model.eval()
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {100*correct/total:.2f}%')


In [35]:

# Ruta al directorio de imágenes convertidas y organizadas
data_directory = 'c:/Users/Matias/OneDrive/Escritorio/Tesis/Data Portugal/converted_images'

# Cargar el dataset
dataset = CustomDataset(root_dir=data_directory, transform=transform)

# Dividir el dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Inicializar el modelo, pérdida y optimizador
n_classes = len(os.listdir(data_directory))  # Asegúrate de que esto refleje el número de clases
model = CNNModel(n_classes=n_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenar el modelo
num_epochs = 10  # Ajusta según sea necesario
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)


Epoch 1/10, Train Loss: 2.6791, Val Loss: 0.7198, Val Acc: 48.26%
Epoch 2/10, Train Loss: 0.6807, Val Loss: 0.7477, Val Acc: 46.33%
Epoch 3/10, Train Loss: 0.6563, Val Loss: 0.7089, Val Acc: 49.81%
Epoch 4/10, Train Loss: 0.6253, Val Loss: 0.7198, Val Acc: 53.28%
Epoch 5/10, Train Loss: 0.5741, Val Loss: 0.7513, Val Acc: 53.28%
Epoch 6/10, Train Loss: 0.5363, Val Loss: 0.7282, Val Acc: 52.51%
Epoch 7/10, Train Loss: 0.4706, Val Loss: 0.7765, Val Acc: 50.58%
Epoch 8/10, Train Loss: 0.3702, Val Loss: 0.8303, Val Acc: 52.90%
Epoch 9/10, Train Loss: 0.2925, Val Loss: 0.9180, Val Acc: 55.60%
Epoch 10/10, Train Loss: 0.2152, Val Loss: 1.0064, Val Acc: 53.67%
